# Практическое задание к Уроку 4. Алгоритм построения дерева решений

In [1]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap
from sklearn import datasets
from sklearn import model_selection
from graphviz import Digraph # если дерево ниже не отображается, требуется установить библиотеку python-graphviz

import numpy as np

#### 1. В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.).

In [3]:
# dot = Digraph(node_attr={'shape': 'box'})
 
# dot.node('A', label='Клиент старше 18 лет?')
# dot.node('B', label='Превышает ли его заработок 50 тысяч рублей?')
# dot.node('C', label='Отказать')
# dot.node('D', label='Были ли у клиента просроченные кредиты ранее?')
# dot.node('E', label='Отказать')
# dot.node('F', label='Отказать')
# dot.node('G', label='Выдать')
 
# dot.edge('A', 'B', label='да')
# dot.edge('A', 'C', label='нет')
# dot.edge('B', 'D', label='да')
# dot.edge('B', 'E', label='нет')
# dot.edge('D', 'F', label='да')
# dot.edge('D', 'G', label='нет')
 
# # dot

In [4]:
# Класс узла
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

    def __str__(self):
        return f'Признак: {self.index}\nПорог: {self.t:.2f}'
        
# Класс терминального узла (листа)
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels  # y_true
        self.prediction = self.predict()  # y_pred
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction

In [5]:
# Расчет критерия Джини
def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1     # "impurity" - "нечистота", степень неопределенности
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [6]:
# Расчет качества
def quality(left_labels, right_labels, current_gini):

    # доля выборки, ушедшей в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0]) # для правого (1-p)
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)


In [7]:
# Разбиение датасета в узле
def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [8]:
# Нахождение наилучшего разбиения
def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5

    current_gini = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        t_values = [row[index] for row in data]
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

**Решение:**

In [9]:
# #ограничение по глубине дерева
# def build_tree(data, labels, level_limit=2, leaf_limit=2, max_level=4):
#     if max_level < 0: raise Exception('Logic errror')
#     quality, t, index = find_best_split(data, labels)

#     if quality == 0 or level_limit or leaf_limit or all(l == labels[0] for l in labels):
#         return Leaf(data, labels)
    
#     true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

#     true_branch  = build_tree(true_data,  true_labels,  max_level - 1)
#     false_branch = build_tree(false_data, false_labels, max_level - 1)

#     # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
#     return Node(index, t, true_branch, false_branch)

In [10]:
leaf_count = 0
level = 0

# Построение дерева с помощью рекурсивной функции
def build_tree(data, labels, leaf_limit=2 ** 32, level_limit=2 ** 32):
    global leaf_count, level
    
    quality, t, index = find_best_split(data, labels)    

    if quality == 0 or leaf_count + 1 == leaf_limit or level == level_limit or all(l == labels[0] for l in labels):        
        return Leaf(data, labels)
        
    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    level += 1    
    
    leaf_count += 1   
    true_branch = build_tree(true_data, true_labels, leaf_limit, level_limit)    
    leaf_count += 1 
    false_branch = build_tree(false_data, false_labels, leaf_limit, level_limit)
    
    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

**Тестирование:**

In [11]:
# сгенерируем данные
classification_data, classification_labels = datasets.make_classification(n_features = 2, n_informative = 2, 
                                                      n_classes = 2, n_redundant=0, 
                                                      n_clusters_per_class=1, random_state=5)

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                                                                     classification_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

# Построим дерево по обучающей выборке
leaf_count = 0
level = 0
my_tree = build_tree(train_data, train_labels, 
                     leaf_limit=4, 
                     level_limit=3)

**Проверка:**

In [12]:
leaf_count = 0

# Напечатаем структуру нашего дерева
def print_tree(node, prev_node):
    global leaf_count
    
    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
#         dot.node(f'leaf {leaf_count}', label=f'leaf {leaf_count}\nПрогноз: {node.prediction}')        
        dot.edge(str(prev_node.t), f'leaf {leaf_count}')
        leaf_count += 1
        return

    # Рекурсионный вызов функции на положительном поддереве
    dot.node(str(node.t), label=str(node))
    if prev_node is not None:
        dot.edge(str(prev_node.t), str(node.t))
        
    print_tree(node.true_branch, node)

    # Рекурсионный вызов функции на отрицательном поддереве
    dot.node(str(node.t), label=str(node))
    print_tree(node.false_branch, node)

dot = Digraph(node_attr={'shape': 'box'})    
print_tree(my_tree, None)   
# dot

In [13]:
leaf_count = 0
level = 0
my_tree = build_tree(train_data, train_labels)

leaf_count = 0

dot = Digraph(node_attr={'shape': 'box'})    
print_tree(my_tree, None)

In [14]:
leaf_count = 0
level = 0
my_tree = build_tree(train_data, train_labels, leaf_limit=2,level_limit=3)

leaf_count = 0

dot = Digraph(node_attr={'shape': 'box'})    
print_tree(my_tree, None)   
# dot

In [15]:
leaf_count = 0
level = 0
my_tree = build_tree(train_data, train_labels, leaf_limit=4, level_limit=2)

leaf_count = 0

dot = Digraph(node_attr={'shape': 'box'})    
print_tree(my_tree, None)   
# dot

Качество ответов на тестовой выборке лучше у дерева с ограничением глубины

#### 2. Для задачи классификации обучить дерево решений с использованием критериев разбиения Джини и Энтропия. Сравнить качество классификации, сделать выводы.

In [16]:
# Расчет критерия Энтропии

def entropy(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 0
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p *np.log2(p)
        
    return impurity

In [17]:
# Расчет качества добавим аргумент, тип оценки

def quality(left_labels, right_labels, current_quality,type_qualitu = 'gini'):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    if type_qualitu == 'entropy':
        return current_quality - p * entropy(left_labels) - (1 - p) * entropy(right_labels)  
    return current_quality - p * gini(left_labels) - (1 - p) * gini(right_labels)

Добавим данный аргумент в другие зависимые функции

In [18]:
# Построим дерево с криетерием энтропии
my_tree = build_tree(train_data, train_labels,type_qualitu = 'entropy')

TypeError: build_tree() got an unexpected keyword argument 'type_qualitu'

In [19]:
# Изучим качество  ответов для обучающей и тестовой выборки по дереву без ограничения c оценкой энтропия
train_answers = predict(train_data, my_tree)
answers = predict(test_data, my_tree)
train_accuracy = tr_nolimit_entropy=accuracy_metric(train_labels, train_answers)
test_accuracy =test_nolimit_entropy= accuracy_metric(test_labels, answers)
print (f'Дерево без ограничений, критерий энтропия: Качество на обучаеющей выборке {train_accuracy}, качество на тесовой выборке {test_accuracy}')

NameError: name 'predict' is not defined

In [20]:
# Построим дерево с критерием Энтропия с лимитом
my_tree_limit = build_tree_limit(train_data, train_labels,5,type_qualitu = 'entropy')

NameError: name 'build_tree_limit' is not defined

In [21]:
# Изучим качество  ответов для обучающей и тестовой выборки по дереву с ограничением по критерю энтропия
train_answers = predict(train_data, my_tree_limit)
answers = predict(test_data, my_tree_limit)
train_accuracy = tr_limit_entropy = accuracy_metric(train_labels, train_answers)
test_accuracy = test_limit_entropy = accuracy_metric(test_labels, answers)
print (f'Дерево c ограниченим по критерю энтропия: Качество на обучаеющей выборке {train_accuracy}, качество на тесовой выборке {test_accuracy}')

NameError: name 'predict' is not defined

In [22]:
result_test = np.array([['критерий','Без ограничений','С ограничением'],['джини',test_nolimit_dgini,test_limit_dgini],
                        ['Энтропия',test_nolimit_entropy,test_limit_entropy]])

NameError: name 'test_nolimit_dgini' is not defined

In [23]:
result_test

NameError: name 'result_test' is not defined

In [24]:
result_train = np.array([['критерий','Без ограничений','С ограничением'],['джини',tr_nolimit_dgini,tr_limit_dgini],
                        ['Энтропия',tr_nolimit_entropy,tr_limit_entropy]])

NameError: name 'tr_nolimit_dgini' is not defined

In [25]:
result_train

NameError: name 'result_train' is not defined

Энтропия дает такие же результаты на тесте, как джини с ограничениме глубины дерева, при этом в энтропии уменьшение глубины не влияет на результат

#### 3 [опция]. Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [26]:
# Класс терминального узла (листа)
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels  # y_true
        self.prediction = self.predict()  # y_pred
        
    def predict(self):
        return np.mean(labels)

In [27]:
# Расчет критерия Джини
def gini(labels):
    impurity = np.var(labels)
        
    return impurity

ИЛИ

In [28]:
def var(y):
    avg = sum(y)/len(y)
    return sum([(yi-avg)**2 for yi in y])/len(y)

def quality(left_labels, right_labels, current_var):
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    return current_var - p * var(left_labels) - (1 - p) * var(right_labels)

In [29]:
def find_best_split(data, labels):
    min_leaf = 5
    current_var = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        t_values = [row[index] for row in data]
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality =  quality(true_labels, false_labels, current_var)
            
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index